In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, desc, collect_list, struct, udf
from pyspark.sql.types import StringType, DoubleType, StructType, StructField, DateType


In [12]:
spark=SparkSession.builder.appName("Ecommerce").getOrCreate()

In [91]:
from pyspark.sql import SparkSession, types
from pyspark.sql.types import *

data=[("1","U1","P1","Plastics",1000.0,"2023-11-01"),
      ("2","U1","P3","Plastics",100.0,"2023-11-02"),
      ("3","U1","P3","Plastics",140.0,"2023-11-02"),
      ("4","U1","P4","Plastics",190.0,"2023-11-05"),
      ("5","U1","P5","Plastics",900.0,"2023-11-10"),
      ("6","U2","P6","Clothing",200.0,"2023-11-11"),
      ("7","U2","P7","Clothing",890.0,"2023-11-12"),
      ("8","U3","P8","Stationary",299.0,"2023-11-12"),
      ("9","U3","P9","Stationary",799.0,"2023-11-13"),
      ("10","U3","P10","Stationary",99.0,"2023-11-14"),]


schema = StructType([
    StructField("transaction_id",StringType(),True),
    StructField("user_id",StringType(),True),
    StructField("product_id",StringType(),True),
    StructField("category",StringType(),True),
    StructField("amount",DoubleType(),True),
    StructField("transaction_date",StringType(),True)
])
df=spark.createDataFrame(data,schema)

In [92]:
df.show()

+--------------+-------+----------+----------+------+----------------+
|transaction_id|user_id|product_id|  category|amount|transaction_date|
+--------------+-------+----------+----------+------+----------------+
|             1|     U1|        P1|  Plastics|1000.0|      2023-11-01|
|             2|     U1|        P3|  Plastics| 100.0|      2023-11-02|
|             3|     U1|        P3|  Plastics| 140.0|      2023-11-02|
|             4|     U1|        P4|  Plastics| 190.0|      2023-11-05|
|             5|     U1|        P5|  Plastics| 900.0|      2023-11-10|
|             6|     U2|        P6|  Clothing| 200.0|      2023-11-11|
|             7|     U2|        P7|  Clothing| 890.0|      2023-11-12|
|             8|     U3|        P8|Stationary| 299.0|      2023-11-12|
|             9|     U3|        P9|Stationary| 799.0|      2023-11-13|
|            10|     U3|       P10|Stationary|  99.0|      2023-11-14|
+--------------+-------+----------+----------+------+----------------+



In [93]:
columns = ["transaction_id","user_id","product_id","category","amount","transaction_date"]
df = spark.createDataFrame(data,columns)

In [111]:
total_spendings_per_user = df.groupBy("user_id").agg(sum("amount").alias("total_spendings_per_user"))

In [112]:
avg_transactions_done = df.groupBy("user_id").agg(avg("amount").alias("avg_transactions_done"))

In [113]:
category_count = df.groupBy("user_id", "category").agg(count("category").alias("category_count"))

In [114]:
window_spec = Window.partitionBy("user_id").orderBy(desc("category_count"))

In [115]:
ranked_category = user_category_count.withColumn("rank",rank().over(window_spec))

In [116]:
favorite_category = ranked_category.filter(col("rank")==1).select("user_id","category")

In [124]:
final_result = total_spendings_per_user.join(avg_transaction,on="user_id",how="inner").join(favorite_category,on="user_id",how="inner")

In [121]:
final_result = final_result.withColumnRenamed("category","favorite_category")

In [122]:
final_result.show()

+-------+------------------------+---------------+-----------------+
|user_id|total_spendings_per_user|avg_transaction|favorite_category|
+-------+------------------------+---------------+-----------------+
|     U1|                  2330.0|          466.0|         Plastics|
|     U2|                  1090.0|          545.0|         Clothing|
|     U3|                  1197.0|          399.0|       Stationary|
+-------+------------------------+---------------+-----------------+

